In [1]:
# imports
import re
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from collections import defaultdict
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

from sklearn.model_selection import train_test_split

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


In [3]:
# use nltk to remove stopwords and lemmatize
# you might need to run: nltk.download() to fetch the stopword package in "all packages"
# you might also need to run ntlk.download("punkt")
english_stopwords = set(stopwords.words("english"))
wordnet_lemmatizer = WordNetLemmatizer()

In [4]:
def clean_input_text(text):
    clean_text = []
    for sent in text:
        clean_sent = ""
        sent_tokens = word_tokenize(sent)
        for token in sent_tokens:
            clean_sent += wordnet_lemmatizer.lemmatize(token) + " " if token not in english_stopwords else ""
        clean_text.append(clean_sent)
    return clean_text

In [5]:
def load_subj_data_and_labels():
    subjective_data_file = "./data/sensitivity/plot.tok.gt9.5000"
    objective_data_file = "./data/sensitivity/quote.tok.gt9.5000"
    
    # Load data from files
    subjective_examples = list(open(subjective_data_file, "r", encoding='utf-8').readlines())
    subjective_examples = [s.strip() for s in subjective_examples]
    objective_examples = list(open(objective_data_file, "r", encoding='utf-8').readlines())
    objective_examples = [s.strip() for s in objective_examples]
    # Split by words
    x_text = subjective_examples + objective_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [1 for _ in subjective_examples]
    negative_labels = [0 for _ in objective_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

x_text, y = load_subj_data_and_labels()
x_text = clean_input_text(x_text)
print(type(x_text))
type(y)

<class 'list'>


numpy.ndarray

In [6]:
# hyperparameters for the cnn dealing with the movie dataset
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.2
NUMBER_DIFFERENT_OUTPUTS =2

In [7]:
# load pretrain glove word2vec instance for preprocessing
filename = './data/glove.6B.300d.txt'

print('Indexing word vectors.')

embeddings_index = {}
with open(filename, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [8]:
# vectorize the input text (both negative and positive )
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(x_text)
sequences = tokenizer.texts_to_sequences(x_text)
word_index = tokenizer.word_index
print(len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
labels[0]

18937
Shape of data tensor: (10000, 1000)
Shape of label tensor: (10000, 2)


array([0., 1.], dtype=float32)

In [9]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]
len(y_val)

2000

In [10]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [11]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [12]:
# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(NUMBER_DIFFERENT_OUTPUTS, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 10s 1ms/step - loss: 0.3854 - acc: 0.8367 - val_loss: 0.3006 - val_acc: 0.8685
Epoch 2/10
8000/8000 [==============================] - 4s 520us/step - loss: 0.2437 - acc: 0.9002 - val_loss: 0.2916 - val_acc: 0.8860
Epoch 3/10
8000/8000 [==============================] - 4s 521us/step - loss: 0.1668 - acc: 0.9356 - val_loss: 0.4257 - val_acc: 0.8360
Epoch 4/10
8000/8000 [==============================] - 4s 521us/step - loss: 0.1015 - acc: 0.9639 - val_loss: 0.4548 - val_acc: 0.8640
Epoch 5/10
8000/8000 [==============================] - 4s 524us/step - loss: 0.0693 - acc: 0.9795 - val_loss: 0.4054 - val_acc: 0.9005
Epoch 6/10
8000/8000 [==============================] - 4s 522us/step - loss: 0.0572 - acc: 0.9844 - val_loss: 0.4400 - val_acc: 0.8920
Epoch 7/10
8000/8000 [==============================] - 4s 526us/step - loss: 0.0810 - acc: 0.9875 - val_loss: 0.5878 - val_acc: 0.8695
E